# Title matching experiment
@author Hieu
@date 30/12/2023

## Description
In this experiment, I matches user query with page title

## Tools
- FAISS
- LaBSE

## Lesson
The sentence transformer model is simply a embedding model that knows the distribution of words. \
It do not have any knowledge on word to a long sentence.\
Therefore, matching a user query to a title is not effective.

## Cause
- not enough information (a title is not enough)
- similar words present in the text cause the score to be high

In [1]:
import pandas as pd

db_df = pd.read_csv("../majors_info.csv")
db_df.head(4)

,Major,Description,Additional Description
0,Bảo hộ lao động,Bảo hộ lao động là ngành học đón đầu xu hướng ...,Bằng cấp\nCử nhân\nKỹ sư Thời gian đào tạo\nTo...
1,Công nghệ kỹ thuật môi trường - Chuyên ngành C...,Công nghệ kỹ thuật môi trường – Chuyên ngành C...,Bằng cấp\nCử nhân\nKỹ sư Thời gian đào tạo\nTo...
2,Công tác xã hội,Công tác xã hội là ngành khoa học vận dụng nhữ...,Bằng cấp\nCử nhân Thời gian đào tạo\nToàn thời...
3,Kế toán,"Kế toán, kiểm toán không chỉ là công cụ quản l...",Bằng cấp\nCử nhân Thời gian đào tạo\nToàn thời...


In [1]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from getpass import getpass

# HUGGING_FACE_KEY = getpass("Hugging face key: ")
HUGGING_FACE_KEY = open("../../hg_key").read()

# Data

In [5]:
MODEL = "sentence-transformers/LaBSE"
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGING_FACE_KEY,
    model_name=MODEL)

In [6]:
db = db_df['Major'].tolist()
faiss_db = FAISS.from_texts(db, embeddings)

# Learn

In [31]:
import numpy as np

a = embeddings.embed_query("Tôi muốn học về lập trình")
test_docs = embeddings.embed_documents(\
    ["kỹ thuật xây dựng", "xây dựng",\
     "phần mềm", "khoa học máy tính", "python", "java",\
     "Phần mềm vẽ kỹ thuật cho thiết kế"])

npa = np.array(a)
for b in test_docs:
    npb = np.array(b)
    distance = np.linalg.norm(npa - npb)
    print(distance)

1.0123114538765887
1.1077420049467213
1.095200175904101
0.9544046579578791
1.2669979125528543
1.163662254111237
0.973616333800583


# RUN

In [7]:
query = "Tôi muốn học về lập trình"
res = faiss_db.similarity_search_with_score(query)
print("Query: "+query)
a = [print(r[0].page_content[:40], "- euclid: ", r[1]) for r in res]

Query: Tôi muốn học về lập trình
Khoa học máy tính - euclid:  0.92115736
Kỹ thuật phần mềm - euclid:  0.9530451
Kỹ thuật xây dựng - euclid:  1.014338
Kỹ thuật xây dựng công trình giao thông - euclid:  1.1019566


In [8]:
query = ""
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].page_content[:40], "- euclid: ", r[1]) for r in res]

Khoa học máy tính - euclid:  0.3378064
Khoa học môi trường - euclid:  0.8905713
Kỹ thuật phần mềm - euclid:  0.94126093
Mạng máy tính và truyền thông dữ liệu - euclid:  0.9495206


In [9]:
query = "Hãy tư vấn ngành công nghệ thông tin"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].page_content[:30], "- euclid: ", r[1]) for r in res]

Khoa học máy tính - euclid:  0.96717566
Kỹ thuật phần mềm - euclid:  1.0049202
Mạng máy tính và truyền thông  - euclid:  1.0302649
Công nghệ kỹ thuật môi trường  - euclid:  1.1638519


In [10]:
query = "Tôi học giỏi môn toán nhất, nếu vậy thì nên theo ngành nào"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].page_content[:30], "- euclid: ", r[1]) for r in res]

Quản lý thể dục thể thao - Chu - euclid:  1.1387161
Việt Nam học - Chuyên ngành Vi - euclid:  1.1471713
Việt Nam học – Chuyên ngành Du - euclid:  1.1800578
Quản trị kinh doanh – Chuyên n - euclid:  1.2042801


In [11]:
query = "Tôi học giỏi môn toán nhất"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].page_content[:30], "- euclid: ", r[1]) for r in res]

Khoa học máy tính - euclid:  1.1363082
Việt Nam học - Chuyên ngành Vi - euclid:  1.2076757
Việt Nam học – Chuyên ngành Du - euclid:  1.2271353
Việt Nam học – Chuyên ngành Du - euclid:  1.2549353


In [13]:
query = "Toán học"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].page_content[:30], "- euclid: ", r[1]) for r in res]

Khoa học máy tính - euclid:  0.47827107
Toán ứng dụng - euclid:  0.5811865
Kế toán - euclid:  0.6899358
Thống kê - euclid:  0.7004967


In [14]:
query = "Tôi có định hướng làm về AI thì nên theo học ngành nào?"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].page_content[:30], "- euclid: ", r[1]) for r in res]

Việt Nam học - Chuyên ngành Vi - euclid:  1.2253768
Việt Nam học – Chuyên ngành Du - euclid:  1.2376243
Quản trị kinh doanh – Chuyên n - euclid:  1.2407134
Quản lý thể dục thể thao - Chu - euclid:  1.2475836


In [18]:
query = "Toán học và thiết kế"
res = faiss_db.similarity_search_with_score(query)
a = [print(r[0].page_content[:30], "- euclid: ", r[1]) for r in res]

Khoa học máy tính - euclid:  0.68308187
Quy hoạch vùng và đô thị - euclid:  0.6861647
Thiết kế đồ họa - euclid:  0.72297275
Kỹ thuật xây dựng - euclid:  0.7528972


In [ ]:
https://github.com/Searchium-ai/hybrid-search